# Q1 - Which states have more/less monitors? (Rank states!)

In [3]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    lines = sc.textFile('epa_hap_daily_summary-small.csv')
    l2 = lines.filter(lambda line: len(line) > 0)
    header = l2.first() #extract header
    #filtrar o header
    non_empty_lines = l2.filter(lambda row : row != header) 

    l3 = non_empty_lines.map(lambda line: line.split(","))
    logRows = l3.map(lambda arr: Row(state=arr[0],county_code=arr[1],site_num=arr[2],coord=arr[5]+arr[6]))
    logRowsDF = spark.createDataFrame(logRows)
    # logRowsDF.printSchema()
    logRowsDF.createOrReplaceTempView("log")
    logRows2DF = spark.sql("SELECT state, count(distinct coord) AS count FROM log GROUP BY state ORDER BY count DESC")
    logRows2DF.show(100)

    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

+-----+-----+
|state|count|
+-----+-----+
|   06|  170|
|   48|  133|
|   27|   94|
|   26|   92|
|   39|   91|
|   36|   67|
|   45|   64|
|   30|   62|
|   42|   61|
|   12|   55|
|   18|   52|
|   08|   51|
|   37|   50|
|   17|   49|
|   53|   43|
|   22|   41|
|   04|   38|
|   20|   37|
|   13|   35|
|   21|   34|
|   41|   32|
|   01|   31|
|   47|   29|
|   55|   26|
|   34|   24|
|   50|   23|
|   40|   22|
|   23|   21|
|   28|   21|
|   25|   19|
|   51|   19|
|   29|   18|
|   19|   18|
|   80|   18|
|   35|   18|
|   16|   17|
|   33|   17|
|   24|   17|
|   09|   15|
|   44|   13|
|   02|   12|
|   49|   12|
|   05|   11|
|   54|   10|
|   32|    9|
|   56|    9|
|   46|    7|
|   38|    7|
|   78|    6|
|   10|    6|
|   31|    6|
|   72|    6|
|   11|    5|
|   15|    5|
+-----+-----+

CPU times: user 658 ms, sys: 111 ms, total: 769 ms
Wall time: 32.9 s


# Q2 - Which counties have the best/worst air quality? (Rank counties considering pollutants’ level!)

In [4]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    lines = sc.textFile('epa_hap_daily_summary-small.csv')
    l2 = lines.filter(lambda line: len(line) > 0)
    header = l2.first() #extract header
    #filtrar o header
    non_empty_lines = l2.filter(lambda row : row != header) 

    l3 = non_empty_lines.map(lambda line: line.split(","))
    logRows = l3.map(lambda arr: Row(State =arr[0], County=arr[1],site_num=arr[2], polutionAvg = float(arr[16])))
    logRowsDF = spark.createDataFrame(logRows)
    # logRowsDF.printSchema()
    logRowsDF.createOrReplaceTempView("log2")
    logRows2DF = spark.sql("SELECT State, County, avg(polutionAvg) AS AveragePolution FROM log2 GROUP BY State, County ORDER BY AveragePolution DESC")
    logRows2DF.show(100)

    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

+-----+------+------------------+
|State|County|   AveragePolution|
+-----+------+------------------+
|   47|   167|            2556.0|
|   36|   059|              19.0|
|   39|   029| 7.385690735785953|
|   30|   067| 5.611212121212121|
|   80|   006|         4.5121875|
|   37|   027| 4.116666666666667|
|   06|   031|3.9843770491803276|
|   06|   039|            3.7393|
|   37|   069|3.3499999999999996|
|   08|   059|              3.07|
|   26|   125| 2.888877848101266|
|   17|   097| 2.879328647058823|
|   12|   031|2.7794603978494625|
|   25|   017|2.6500000000000004|
|   20|   093|2.3753333333333333|
|   42|   017|2.3674999999999997|
|   06|   079|2.3333333333333335|
|   37|   065|             2.325|
|   20|   145|2.2941176470588234|
|   36|   119|          2.239375|
|   37|   101|             2.225|
|   09|   003|2.0787055896226416|
|   37|   077|2.0285714285714285|
|   53|   003|             2.025|
|   37|   061|               2.0|
|   08|   013| 1.960470901234568|
|   05|   035|

# Q3 - Which states have the best/worst air quality in each year? (Rank states per year considering pollutants' levels!)

In [5]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    epa_daily = sc.textFile('epa_hap_daily_summary-small.csv')

    non_empty_lines_epa_daily = epa_daily.filter( lambda line : len(line) > 0 )

    header_epa_daily = non_empty_lines_epa_daily.first() #extract header
    non_empty_lines_epa_daily = non_empty_lines_epa_daily.filter(lambda row : row != header_epa_daily)  

    epa_daily = non_empty_lines_epa_daily.map(lambda line: line.split(','))
    
    #cria tuplos (ano,(estado,ar))
    epa_daily=epa_daily.map(lambda coord: Row(year = coord[11].split('-')[0], state = coord[24], airq = float(coord[16])))
    
    epa_dailyDF = spark.createDataFrame(epa_daily)
    

    epa_dailyDF.createOrReplaceTempView("log")
    
    query= "SELECT year,state,round(avg(airq),6) AS AveragePolution FROM log GROUP BY year,state ORDER BY year,AveragePolution"
    
    epa_daily2DF = spark.sql(query)
    epa_daily2DF.show(100)
    
    sc.stop()
    
except Exception as err:
    print (err)
    sc.stop()

+----+--------------------+---------------+
|year|               state|AveragePolution|
+----+--------------------+---------------+
|1990|           Wisconsin|            0.0|
|1990|       West Virginia|            0.0|
|1990|      Virgin Islands|            0.0|
|1990|            Oklahoma|            0.0|
|1990|              Hawaii|        1.97E-4|
|1990|              Nevada|        4.21E-4|
|1990|              Alaska|        4.42E-4|
|1990|        South Dakota|        5.71E-4|
|1990|          Washington|        5.97E-4|
|1990|             Wyoming|        6.05E-4|
|1990|                Utah|        7.97E-4|
|1990|          New Mexico|        8.22E-4|
|1990|              Oregon|         8.6E-4|
|1990|             Arizona|        8.62E-4|
|1990|               Maine|        9.79E-4|
|1990|            Colorado|       0.002162|
|1990|         Mississippi|       0.002667|
|1990|            Missouri|         0.0056|
|1990|            Michigan|        0.00656|
|1990|         Connecticut|     

# Q4 - For each state, what is the average distance (in km) of the monitors in that state to the state center? For simplicity, assume that 1 degree of latitude or logitude equals to 111 km. (Monitor dispersion per state!)

In [6]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext
try:
    epa_daily = sc.textFile('epa_hap_daily_summary-small.csv')
    usa_state = sc.textFile('usa_states.csv')
    
    usa_state = sc.textFile('usa_states.csv')

    non_empty_lines_epa_daily = epa_daily.filter( lambda line : len(line) > 0 )
    non_empty_lines_usa_state = usa_state.filter( lambda line : len(line) > 0 )

    header_epa_daily = non_empty_lines_epa_daily.first() #extract header
    header_usa_state = non_empty_lines_usa_state.first() #extract header



    non_empty_lines_epa_daily = non_empty_lines_epa_daily.filter(lambda row : row != header_epa_daily)  
    non_empty_lines_usa_state = non_empty_lines_usa_state.filter(lambda row : row != header_usa_state)  

    epa_daily = non_empty_lines_epa_daily.map(lambda line: line.split(','))
    usa_state = non_empty_lines_usa_state.map(lambda line: line.split(','))
    
    table_epa_daily=epa_daily.map(lambda arr: Row(state=arr[24],lat=float(arr[5])*111, long=float(arr[6])*111))
    table_usa_state=usa_state.map(lambda arr: Row(state=arr[1],lat=(float(arr[2])*111+float(arr[3])*111)/2, long=(float(arr[4])*111+float(arr[5])*111)/2))
    
    table_epa_daily = spark.createDataFrame(table_epa_daily)
    table_usa_state = spark.createDataFrame(table_usa_state)
    
    table_epa_daily.createOrReplaceTempView("table_epa_daily")
    table_usa_state.createOrReplaceTempView("table_usa_state")
    
    query="SELECT table.state,AVG(table.distance) AS avgDistance FROM (SELECT DISTINCT table_epa_daily.state AS state, SQRT(POWER(FLOAT(table_epa_daily.lat)-FLOAT(table_usa_state.lat),2)+POWER(FLOAT(table_epa_daily.long)-FLOAT(table_usa_state.long),2)) AS distance FROM table_epa_daily INNER JOIN table_usa_state ON table_epa_daily.state=table_usa_state.state) table GROUP BY table.state ORDER BY avgDistance DESC"    
    new_table=spark.sql(query)
    new_table.show()
    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

+------------+------------------+
|       state|       avgDistance|
+------------+------------------+
|    Virginia| 715.4327448133683|
|      Alaska| 603.6994841759217|
|       Texas| 512.1842175513935|
|     Vermont|  504.063097911535|
|    Illinois|440.85400685379017|
|South Dakota| 365.8450815332589|
|     Florida| 336.5449260322615|
|  California| 328.2265897979527|
|    Michigan| 326.4119709194728|
|      Nevada|  326.281231608471|
|    Nebraska| 307.1412258440534|
|      Kansas|292.07954678193937|
|       Idaho|289.63520486710013|
|     Montana|286.83825138140065|
|    New York| 283.7273444433415|
|     Wyoming|283.64057827794375|
|      Oregon|  268.853544089414|
|Pennsylvania|251.41513622817573|
|North Dakota|248.42175081028077|
|    Oklahoma| 236.8822819002025|
+------------+------------------+
only showing top 20 rows



# Q5 - How many sensors there are per quadrant (NW, NE, SE, SW) in each state? To answer this question, you should approximate each state’s area to a rectangle as defined in the file “usa_satates.csv”, and divide that area in 4 quadrants (NW, NE, SE, SW). (Count monitors per sate qudrant!)

In [7]:
%%time
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    
    ########## Ter USA States com latitude e longitude medias ########## 

    usaStates = sc.textFile('usa_states.csv')
    
    non_empty_linesStates = usaStates.filter( lambda line : len(line) > 0 )    
    header = non_empty_linesStates.first() #extract header
    #filtrar o header
    non_empty_linesStates = non_empty_linesStates.filter(lambda row : row != header)  
    wordsUSA = non_empty_linesStates.map( lambda line : line.split(','))

    linhasUSA = wordsUSA.map(lambda arr: Row(stateNameUSA=arr[1] ,avgLat = (float(arr[2])+float(arr[3]))/2 , avgLong = (float(arr[4])+float(arr[5]))/2))

    linhasUSADF = spark.createDataFrame(linhasUSA)
    linhasUSADF.createOrReplaceTempView("logUSA")

    #######################

    epaFile = sc.textFile('epa_hap_daily_summary-small.csv')
    
    non_empty_linesEPA = epaFile.filter( lambda line : len(line) > 0 )    
    header = non_empty_linesEPA.first() #extract header
    #filtrar o header
    non_empty_linesEPA = non_empty_linesEPA.filter(lambda row : row != header)  
    wordsEPA = non_empty_linesEPA.map( lambda line : line.split(','))

    linhasEPA = wordsEPA.map(lambda arr: Row(stateName=arr[24] , lat = float(arr[5]) , long = float(arr[6])))
    linhasEPADF = spark.createDataFrame(linhasEPA)
    linhasEPADF.createOrReplaceTempView("logEPA")
    
    logRowsEPA = spark.sql("SELECT COUNT (DISTINCT stateName, lat, long) as NrMonitors, stateName,\
                        CASE WHEN ((lat <= avgLat) AND (long <= avgLong)) THEN 'NW' \
                            WHEN ((lat <= avgLat) AND (long > avgLong)) THEN 'SW' \
                            WHEN ((lat > avgLat) AND (long <= avgLong)) THEN 'NE' \
                            WHEN ((lat > avgLat) AND (long > avgLong)) THEN 'SE' \
                        END as Quadrant\
                        FROM logUSA INNER JOIN logEPA ON logUSA.stateNameUSA = logEPA.stateName\
                        GROUP BY stateName, Quadrant\
                        ORDER BY NrMonitors DESC")
                           
    logRowsEPA.show(10000)

    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

+----------+--------------+--------+
|NrMonitors|     stateName|Quadrant|
+----------+--------------+--------+
|        84|    California|      NE|
|        74|      Michigan|      SW|
|        72|         Texas|      SW|
|        68|    California|      SW|
|        50|     Minnesota|      SW|
|        43|      New York|      SW|
|        36|          Ohio|      NW|
|        36|  Pennsylvania|      SW|
|        36|       Montana|      NW|
|        35|     Louisiana|      SW|
|        34|         Texas|      SE|
|        33|South Carolina|      NE|
|        32|      Illinois|      SE|
|        30|          Ohio|      SE|
|        27|       Florida|      SE|
|        26|North Carolina|      SE|
|        25|      Colorado|      SE|
|        24|         Texas|      NE|
|        23|       Florida|      SW|
|        21|     Wisconsin|      SW|
|        21|       Georgia|      NE|
|        21|     Minnesota|      NW|
|        20|    Washington|      NE|
|        19|North Carolina|      NE|
|